# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [44]:
# import libraries
import pandas as pd
import sqlite3
import sqlalchemy
from sqlalchemy import create_engine

import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from sklearn.model_selection import GridSearchCV
from sklearn.datasets import make_multilabel_classification
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, f1_score, classification_report, make_scorer, fbeta_score

import numpy as np

import re


nltk.download(['punkt', 'wordnet'])

Requirement already up-to-date: scikit-learn in /opt/conda/lib/python3.6/site-packages (0.24.2)
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [27]:
# load data from database
engine = create_engine('sqlite:///InsertDatabaseName.db')
df = pd.read_sql_table("Clean_Messages", con=engine)
print(df.head())

#X = df['message']
#Y = df.drop(columns=['id','message','original','genre'])

X = df['message']
Y = df.drop(['id','message','original', 'genre'], axis = 1)


   id                                            message  \
0   2  Weather update - a cold front from Cuba that c...   
1   7            Is the Hurricane over or is it not over   
2   8                    Looking for someone but no name   
3   9  UN reports Leogane 80-90 destroyed. Only Hospi...   
4  12  says: west side of Haiti, rest of the country ...   

                                            original   genre  related  \
0  Un front froid se retrouve sur Cuba ce matin. ...  direct        1   
1                 Cyclone nan fini osinon li pa fini  direct        1   
2  Patnm, di Maryani relem pou li banm nouvel li ...  direct        1   
3  UN reports Leogane 80-90 destroyed. Only Hospi...  direct        1   
4  facade ouest d Haiti et le reste du pays aujou...  direct        1   

   request  offer  aid_related  medical_help  medical_products      ...        \
0        0      0            0             0                 0      ...         
1        0      0            1        

In [28]:
Y.shape

(26216, 36)

In [29]:
# Get the label names
category_names = Y.columns
category_names

Index(['related', 'request', 'offer', 'aid_related', 'medical_help',
       'medical_products', 'search_and_rescue', 'security', 'military',
       'child_alone', 'water', 'food', 'shelter', 'clothing', 'money',
       'missing_people', 'refugees', 'death', 'other_aid',
       'infrastructure_related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure',
       'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold',
       'other_weather', 'direct_report'],
      dtype='object')

In [30]:
X.head()

0    Weather update - a cold front from Cuba that c...
1              Is the Hurricane over or is it not over
2                      Looking for someone but no name
3    UN reports Leogane 80-90 destroyed. Only Hospi...
4    says: west side of Haiti, rest of the country ...
Name: message, dtype: object

In [31]:
Y.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [32]:
#def tokenize(text):
    #""" This function tokenizes text"""
    
    #tokens = word_tokenize(text)
    #lemmatizer = WordNetLemmatizer()
    
    #cleaned_tokens=[]
    #for token in tokens:
        #clean_token = lemmatizer.lemmatize(token).lower().strip()
        #cleaned_tokens.append(clean_token)
        
    #return cleaned_tokens



def tokenize(text):
    
    text = re.sub(r"[^a-zA-Z0-9]", " ", text)
    tokens=word_tokenize(text)
    stopwords = stopwords.tokens('English')
    lemmatizar = WordNetLemmatizer()
    cleaned_tokens = []
    
    for tok in tokens:
        clean_token = lemmatizer.lemmatize(tok).lower().strip()
        cleaned_tokens.append(clean_token)
        
    return cleaned_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [33]:
def model_pipeline():
    pipeline = Pipeline([
        ('vect', CountVectorizer()),
        ('tfidf', TfidfTransformer()),
        ('clf',  MultiOutputClassifier(RandomForestClassifier()))
    ])

    return pipeline


In [34]:
pipeline = Pipeline([
        ('vect', CountVectorizer()),
        ('tfidf', TfidfTransformer()),
        ('clf',  MultiOutputClassifier(RandomForestClassifier()))
    ])

In [35]:
pipeline.get_params().keys()


dict_keys(['memory', 'steps', 'vect', 'tfidf', 'clf', 'vect__analyzer', 'vect__binary', 'vect__decode_error', 'vect__dtype', 'vect__encoding', 'vect__input', 'vect__lowercase', 'vect__max_df', 'vect__max_features', 'vect__min_df', 'vect__ngram_range', 'vect__preprocessor', 'vect__stop_words', 'vect__strip_accents', 'vect__token_pattern', 'vect__tokenizer', 'vect__vocabulary', 'tfidf__norm', 'tfidf__smooth_idf', 'tfidf__sublinear_tf', 'tfidf__use_idf', 'clf__estimator__bootstrap', 'clf__estimator__class_weight', 'clf__estimator__criterion', 'clf__estimator__max_depth', 'clf__estimator__max_features', 'clf__estimator__max_leaf_nodes', 'clf__estimator__min_impurity_decrease', 'clf__estimator__min_impurity_split', 'clf__estimator__min_samples_leaf', 'clf__estimator__min_samples_split', 'clf__estimator__min_weight_fraction_leaf', 'clf__estimator__n_estimators', 'clf__estimator__n_jobs', 'clf__estimator__oob_score', 'clf__estimator__random_state', 'clf__estimator__verbose', 'clf__estimator__

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [36]:
X_train, X_test, y_train, y_test = train_test_split(X,Y)

In [37]:
model = model_pipeline()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)


### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [38]:
y_pred_df = pd.DataFrame(y_pred, columns = y_test.columns)
y_pred_df.head()


,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [39]:
for column in y_test.columns:
    print('column_name: {}\n'.format(column))
    print(classification_report(y_test[column], y_pred_df[column]))

column_name: related

             precision    recall  f1-score   support

          0       0.62      0.35      0.45      1534
          1       0.82      0.93      0.87      4969
          2       0.50      0.47      0.48        51

avg / total       0.77      0.79      0.77      6554

column_name: request

             precision    recall  f1-score   support

          0       0.89      0.98      0.93      5448
          1       0.80      0.42      0.55      1106

avg / total       0.88      0.88      0.87      6554

column_name: offer

             precision    recall  f1-score   support

          0       0.99      1.00      1.00      6521
          1       0.00      0.00      0.00        33

avg / total       0.99      0.99      0.99      6554

column_name: aid_related

             precision    recall  f1-score   support

          0       0.72      0.86      0.79      3803
          1       0.74      0.55      0.63      2751

avg / total       0.73      0.73      0.72      655

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [40]:
params = pipeline.get_params()
params

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=None, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
               oob_score=False, random_state=None, verbose=0,
               warm_s

In [41]:
# create a function to put all the F1 score for each output. The score should be the median F1 score.

def f1_metric(y_true, y_pred):
    f1_list = []
    for i in range(np.shape(y_pred)[1]):
        f1 = f1_score(np.array(y_true)[:, i], y_pred[:, i], average= 'micro')
        f1_list.append(f1)
        
    score = np.median(f1_list)
    return score

In [42]:
def model_pipeline_cv():
    pipeline = Pipeline([
        ('vect', CountVectorizer()),
        ('tfidf', TfidfTransformer()),
        ('clf',  MultiOutputClassifier(RandomForestClassifier()))
    ])
    
    parameters = {
        'vect__max_df': (0.5, 0.75),
        'clf__estimator__max_depth': [10, 20],
        'vect__ngram_range' : ((1, 1), (1,2)),
        'clf__estimator__n_estimators': [50, 100]
    }
    
    scorer = make_scorer(f1_metric,greater_is_better = True)
    
    cv = GridSearchCV(pipeline, param_grid = parameters, scoring= scorer, verbose= 5, n_jobs=1)

    return cv



In [45]:
model_grid = model_pipeline_cv()
model_grid.fit(X_train, y_train)


Fitting 3 folds for each of 16 candidates, totalling 48 fits
[CV] clf__estimator__max_depth=10, clf__estimator__n_estimators=50, vect__max_df=0.5, vect__ngram_range=(1, 1) 
[CV]  clf__estimator__max_depth=10, clf__estimator__n_estimators=50, vect__max_df=0.5, vect__ngram_range=(1, 1), score=0.9536161122978334, total=  12.7s
[CV] clf__estimator__max_depth=10, clf__estimator__n_estimators=50, vect__max_df=0.5, vect__ngram_range=(1, 1) 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   15.9s remaining:    0.0s


[CV]  clf__estimator__max_depth=10, clf__estimator__n_estimators=50, vect__max_df=0.5, vect__ngram_range=(1, 1), score=0.9543790051876716, total=  12.6s
[CV] clf__estimator__max_depth=10, clf__estimator__n_estimators=50, vect__max_df=0.5, vect__ngram_range=(1, 1) 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   31.7s remaining:    0.0s


[CV]  clf__estimator__max_depth=10, clf__estimator__n_estimators=50, vect__max_df=0.5, vect__ngram_range=(1, 1), score=0.953768690875801, total=  12.9s
[CV] clf__estimator__max_depth=10, clf__estimator__n_estimators=50, vect__max_df=0.5, vect__ngram_range=(1, 2) 


...

[CV]  clf__estimator__max_depth=10, clf__estimator__n_estimators=50, vect__max_df=0.5, vect__ngram_range=(1, 2), score=0.9536161122978334, total=  20.2s
[CV] clf__estimator__max_depth=10, clf__estimator__n_estimators=50, vect__max_df=0.5, vect__ngram_range=(1, 2) 


[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:  1.3min remaining:    0.0s


[CV]  clf__estimator__max_depth=10, clf__estimator__n_estimators=50, vect__max_df=0.5, vect__ngram_range=(1, 2), score=0.9543790051876716, total=  20.1s
[CV] clf__estimator__max_depth=10, clf__estimator__n_estimators=50, vect__max_df=0.5, vect__ngram_range=(1, 2) 
[CV]  clf__estimator__max_depth=10, clf__estimator__n_estimators=50, vect__max_df=0.5, vect__ngram_range=(1, 2), score=0.953768690875801, total=  20.6s
[CV] clf__estimator__max_depth=10, clf__estimator__n_estimators=50, vect__max_df=0.75, vect__ngram_range=(1, 1) 
[CV]  clf__estimator__max_depth=10, clf__estimator__n_estimators=50, vect__max_df=0.75, vect__ngram_range=(1, 1), score=0.9536161122978334, total=  12.7s
[CV] clf__estimator__max_depth=10, clf__estimator__n_estimators=50, vect__max_df=0.75, vect__ngram_range=(1, 1) 
[CV]  clf__estimator__max_depth=10, clf__estimator__n_estimators=50, vect__max_df=0.75, vect__ngram_range=(1, 1), score=0.9543790051876716, total=  12.8s
[CV] clf__estimator__max_depth=10, clf__estimator

[Parallel(n_jobs=1)]: Done  48 out of  48 | elapsed: 32.8min finished


GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'vect__max_df': (0.5, 0.75), 'clf__estimator__max_depth': [10, 20], 'vect__ngram_range': ((1, 1), (1, 2)), 'clf__estimator__n_estimators': [50, 100]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=make_scorer(f1_metric), verbose=5)

In [46]:
y_pred_grid = model_grid.predict(X_test)

In [47]:
y_pred_grid

array([[1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ..., 
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0]])

In [49]:
model_grid.best_params_

{'clf__estimator__max_depth': 10,
 'clf__estimator__n_estimators': 50,
 'vect__max_df': 0.5,
 'vect__ngram_range': (1, 1)}

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [51]:
predictions = model_grid.predict(X_test)

In [52]:
y_pred_df = pd.DataFrame(predictions, columns = y_test.columns)
y_pred_df.head()


,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [53]:
for column in y_test.columns:
    print('column_name: {}\n'.format(column))
    print(classification_report(y_test[column], y_pred_df[column]))


column_name: related

             precision    recall  f1-score   support

          0       0.00      0.00      0.00      1534
          1       0.76      1.00      0.86      4969
          2       0.00      0.00      0.00        51

avg / total       0.57      0.76      0.65      6554

column_name: request

             precision    recall  f1-score   support

          0       0.83      1.00      0.91      5448
          1       0.00      0.00      0.00      1106

avg / total       0.69      0.83      0.75      6554

column_name: offer

             precision    recall  f1-score   support

          0       0.99      1.00      1.00      6521
          1       0.00      0.00      0.00        33

avg / total       0.99      0.99      0.99      6554

column_name: aid_related

             precision    recall  f1-score   support

          0       0.60      1.00      0.75      3803
          1       0.94      0.07      0.14      2751

avg / total       0.74      0.61      0.49      655

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [54]:
# Trying out another model

from sklearn.tree import DecisionTreeClassifier


pipeline_svc = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf',  MultiOutputClassifier(DecisionTreeClassifier()))
    ])

pipeline_svc.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=None, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
               max_features=None, max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, presort=False, random_state=None,
               splitter='best'),
              n_jobs=1))],
 'vect': CountVectorizer(analyzer=

In [55]:
def model_pipeline_2():
    pipeline_svc = Pipeline([
        ('vect', CountVectorizer()),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(DecisionTreeClassifier()))
    ])
    
    parameters_svc = {
        'tfidf__use_idf':[True, False],
        'clf__estimator__max_depth' : [4,8,10]
    }
    
    scorer = make_scorer(f1_metric, greater_is_better = True)
    
    cv = GridSearchCV(pipeline_svc, param_grid = parameters_svc, scoring= scorer, verbose= 5)

    return cv



In [56]:
model_grid2 = model_pipeline_2()
model_grid2.fit(X_train, y_train)
y_pred_grid2 = model_grid2.predict(X_test)

Fitting 3 folds for each of 6 candidates, totalling 18 fits
[CV] clf__estimator__max_depth=4, tfidf__use_idf=True ................
[CV]  clf__estimator__max_depth=4, tfidf__use_idf=True, score=0.9616264876411351, total=   6.8s
[CV] clf__estimator__max_depth=4, tfidf__use_idf=True ................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    7.3s remaining:    0.0s


[CV]  clf__estimator__max_depth=4, tfidf__use_idf=True, score=0.9636100091547146, total=   6.7s
[CV] clf__estimator__max_depth=4, tfidf__use_idf=True ................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   14.6s remaining:    0.0s


[CV]  clf__estimator__max_depth=4, tfidf__use_idf=True, score=0.9644491913335367, total=   6.8s
[CV] clf__estimator__max_depth=4, tfidf__use_idf=False ...............


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:   21.9s remaining:    0.0s


[CV]  clf__estimator__max_depth=4, tfidf__use_idf=False, score=0.9619316447970705, total=   5.9s
[CV] clf__estimator__max_depth=4, tfidf__use_idf=False ...............


[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:   28.4s remaining:    0.0s


[CV]  clf__estimator__max_depth=4, tfidf__use_idf=False, score=0.9642203234665854, total=   5.8s
[CV] clf__estimator__max_depth=4, tfidf__use_idf=False ...............
[CV]  clf__estimator__max_depth=4, tfidf__use_idf=False, score=0.964372902044553, total=   5.9s
[CV] clf__estimator__max_depth=8, tfidf__use_idf=True ................
[CV]  clf__estimator__max_depth=8, tfidf__use_idf=True, score=0.963457430576747, total=  13.3s
[CV] clf__estimator__max_depth=8, tfidf__use_idf=True ................
[CV]  clf__estimator__max_depth=8, tfidf__use_idf=True, score=0.9637625877326823, total=  13.4s
[CV] clf__estimator__max_depth=8, tfidf__use_idf=True ................
[CV]  clf__estimator__max_depth=8, tfidf__use_idf=True, score=0.9635337198657309, total=  13.4s
[CV] clf__estimator__max_depth=8, tfidf__use_idf=False ...............
[CV]  clf__estimator__max_depth=8, tfidf__use_idf=False, score=0.9637625877326823, total=  11.7s
[CV] clf__estimator__max_depth=8, tfidf__use_idf=False .............

[Parallel(n_jobs=1)]: Done  18 out of  18 | elapsed:  3.6min finished


In [58]:
y_pred_df_grid2 = pd.DataFrame(y_pred_grid2, columns = y_test.columns)
for column in y_test.columns:
    print('column_name: {}\n'.format(column))
    print(classification_report(y_test[column], y_pred_df_grid2[column]))


column_name: related

             precision    recall  f1-score   support

          0       0.67      0.19      0.29      1534
          1       0.79      0.97      0.87      4969
          2       0.00      0.00      0.00        51

avg / total       0.75      0.78      0.73      6554

column_name: request

             precision    recall  f1-score   support

          0       0.90      0.97      0.93      5448
          1       0.73      0.45      0.56      1106

avg / total       0.87      0.88      0.87      6554

column_name: offer

             precision    recall  f1-score   support

          0       0.99      1.00      1.00      6521
          1       0.00      0.00      0.00        33

avg / total       0.99      0.99      0.99      6554

column_name: aid_related

             precision    recall  f1-score   support

          0       0.70      0.85      0.77      3803
          1       0.71      0.50      0.59      2751

avg / total       0.70      0.70      0.69      655

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 9. Export your model as a pickle file

In [ ]:
def save_model(model, model_filepath):
    """Save the model into pickle object"""

    # Save the model based on model_filepath given
    pkl_filename = '{}'.format(model_filepath)
    with open(pkl_filename, 'wb') as file:
        pickle.dump(model, file)


In [59]:
import pickle
filename = 'ML_pipeline_first_Model.sav'
pickle.dump(model, open(filename, 'wb'))

In [60]:
filename = 'ML_pipeline_second_Model.sav'
pickle.dump(model_grid, open(filename, 'wb'))

In [61]:
filename = 'ML_pipeline_third_Model.sav'
pickle.dump(model_grid2, open(filename, 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.